In [1]:
%%capture
sys
sys.path.insert(0,'/Applications/IBMSPSSStatistics/Subscription/Python3/lib/python3.8/site-packages')
import spss
from scipy import stats

In [2]:
import os
savPath = os.path.join(os.getcwd(), 'Michael Working May 17.sav')
spss.Submit("GET FILE '{0}'.".format(savPath))
print(spss.GetVariableCount())

198


In [3]:
# Extract variable information from SPSS

NameToIndexMap = {}
NameToLabelMap = {}
varcount=spss.GetVariableCount()
for i in range(varcount):
    name = spss.GetVariableName(i)
    NameToIndexMap[name] = i
    NameToLabelMap[name] = spss.GetVariableLabel(i)


In [4]:
# Fix up a few issues in the data. Typically Yes/No answers in the study should have
# the values 1=yes and 2=no

def remapQ97_PartnerSpecialistHelp(x):
    return 2 if x == None else x - 22

def remapOptionalYesNo(x):
    return 2 if x == None else x

VariableMappings = {
    'Q97_PartnerSpecialistHelp': remapQ97_PartnerSpecialistHelp,
    'Q47': remapOptionalYesNo,
    'Q55': remapOptionalYesNo,
    'Q59': remapOptionalYesNo,
    'Q63': remapOptionalYesNo,
    'Q67': remapOptionalYesNo,
    'Q72': remapOptionalYesNo,
    'Q76': remapOptionalYesNo,
    'Q80': remapOptionalYesNo,
}

In [5]:
def TauC(var1, var2):
    """
    Compute the Kendall's Tau-C value for two study variables
    """
    index1 = NameToIndexMap[var1]
    index2 = NameToIndexMap[var2]
    cursor1 = spss.Cursor([index1])
    data1 = cursor1.fetchall()
    data1 = [x[0] for x in cursor1.fetchall()]
    cursor1.close()
    cursor2 = spss.Cursor([index2])
    data2 = [x[0] for x in cursor2.fetchall()]
    cursor2.close()
    if var1 in VariableMappings:
        data1 = [VariableMappings[var1](x) for x in data1]
    if var2 in VariableMappings:
        data2 = [VariableMappings[var2](x) for x in data2]
    return stats.kendalltau(data1, data2, variant='c')


In [6]:
import numpy
import pandas as pandas

def ChiSquared(var1, var2):
    """
    Compute the Chi squared value for two study variables
    """
    index1 = NameToIndexMap[var1]
    index2 = NameToIndexMap[var2]
    cursor1 = spss.Cursor([index1])
    data1 = [x[0] for x in cursor1.fetchall()]
    cursor1.close()
    cursor2 = spss.Cursor([index2])
    data2 = [x[0] for x in cursor2.fetchall()]
    cursor2.close()
    if var1 in VariableMappings:
        data1 = [VariableMappings[var1](x) for x in data1]
    if var2 in VariableMappings:
        data2 = [VariableMappings[var2](x) for x in data2]

    data = numpy.array([data1, data2])
    data = pandas.DataFrame({var1: data1, var2: data2})

    # Contingency table.
    contingency = pandas.crosstab(data[var1], data[var2])
    return stats.chi2_contingency(contingency, correction=False)

def CramersV(var1, var2):
    X2, p, dof, expected = ChiSquared(var1, var2)
    n = spss.GetCaseCount()

    #calculate Cramer's V 
    V = numpy.sqrt((X2/n) / dof)
    return V, X2, p

In [7]:
import math

def heatMapColour(value):
  """
  Compute heatmap colours for a value between 0 and 1
  """
  colour1 = { 'r': 255, 'g': 255, 'b': 255 }
  colour2 = { 'r': 255, 'g': 0, 'b': 0 }
  colour3 = { 'r': 100, 'g': 0, 'b': 0 }

  p1 = (1 - (value * 2)) if (value <= 0.5) else 0
  p2 = (value * 2) if (value <= 0.5) else 1 - ((value - 0.5) * 2)
  p3 = 0 if (value <= 0.5) else ((value - 0.5) * 2)
  
  blended = {
    'r': math.floor(colour1['r'] * p1 + colour2['r'] * p2 + colour3['r'] * p3),
    'g': math.floor(colour1['g'] * p1 + colour2['g'] * p2 + colour3['g'] * p3),
    'b': math.floor(colour1['b'] * p1 + colour2['b'] * p2 + colour3['b'] * p3),
  }

  return {
    'color': 'rgb(0, 0, 0)',
    'background': f'rgb({blended["r"]}, {blended["g"]}, {blended["b"]})'
  }

In [8]:
from yattag import Doc

significantColours = {
    '< 0.01': {
      'colour': 'rgb(255, 255, 255)',
      'background': 'rgb(8, 96, 95)'
    },
    '< 0.05': {
      'colour': 'rgb(0, 0, 0)',
      'background': 'rgb(54, 201, 198)'
    },
    '>= 0.05': {
      'colour': 'rgb(0, 0, 0)',
      'background': 'rgb(255, 255, 255)'
    }
}

def significanceColour(value):
    if value < 0.01:
        return significantColours['< 0.01']
    if value < 0.05:
        return significantColours['< 0.05']
    return significantColours['>= 0.05']

significanceKeyData = [
    {
        'label': '𝒑 < 0.01',
        'colour': significantColours['< 0.01']['colour'],
        'background': significantColours['< 0.01']['background'],
    },
    {
        'label': '𝒑 < 0.05',
        'colour': significantColours['< 0.05']['colour'],
        'background': significantColours['< 0.05']['background'],
    },
    {
        'label': '𝒑 >= 0.05',
        'colour': significantColours['>= 0.05']['colour'],
        'background': significantColours['>= 0.05']['background'],
    },
]

doc, tag, text = Doc().tagtext()
with tag('div', klass='key', style='padding-top: 2rem;'):
    for item in significanceKeyData:
        with tag('span', klass='key-item', style=f'border: 1px solid black; padding: 1em; color: {item["colour"]};background-color: {item["background"]};margin-right: 1em'):
            text(item['label'])
significanceKey = doc.getvalue()


In [9]:

from yattag import Doc
from yattag import indent

def computeTable(selectedVariables, computeCellFn, formatCellFn, colourCellFn, labels):
    """
    Compute an HTML table of statistical values where the statistical computation is
    run on each pair of variables
    """
    doc, tag, text = Doc().tagtext()

    computedData = [[computeCellFn(row, column) for column in selectedVariables] for row in selectedVariables]

    with tag('style'):
        doc.asis('''
            table {
                font-family: Arial, Helvetica, sans-serif;
                border-collapse: collapse;
            }
            td {
                color: black;
                border: 1px solid black;
            }
            thead th {
                padding-bottom: 1em;
            }
            thead th div {
                white-space: nowrap;
            }
            tbody th {
                text-align: right;
                padding-right: 1em;
            }
        ''')
    with tag('table'):
        with tag('thead'):
            with tag('tr'):
                doc.stag('th')
                for label in labels:
                    with tag('th', style='vertical-align: bottom;'):
                        with tag('div', style='writing-mode: vertical-rl;'):
                            text(label)
        with tag('tbody'):
            for rowVar, rowData, label in zip(selectedVariables, computedData, labels):
                with tag('tr'):
                    with tag('th'):
                        text(label)
                    for columnVar, cellData in zip(selectedVariables, rowData):
                        if columnVar == rowVar:
                            doc.stag('td')
                        else:
                            colour = colourCellFn(cellData)
                            with tag('td', style=f'background: { colour["background"] };color: { colour["colour"] };'):
                                doc.asis(formatCellFn(cellData))

    return indent(doc.getvalue())


In [10]:
from IPython.display import HTML, display
from yattag import Doc
import urllib

def displayTable(html, title=None):
    if title != None:
        html = f'<h2>{title}</h2>{html}'
    display(HTML(html))

def writeTable(html, name):
    with open(f'output/{name}', 'w', encoding='utf-8') as writer:
        writer.write(html)


In [11]:
def TauTable(variables, labels):
    """
    Generate an HTML heat map table of Kendell's Tau-C computations for each variable 
    against each other variable. This gives a quick visual overview of where there are
    correlations.
    """
    def formatCell(value):
        doc, tag, text, line = Doc().ttl()
        with tag('div', klass='value'):
            text('Tau-c={0:.2g}'.format(value.correlation))
        with tag('div', klass='p-value'):
            text('𝒑<{0:.3g}'.format(value.pvalue))
        return doc.getvalue()
    def colourCell(value):
        if value != None:
            return { 'background': heatMapColour(abs(value.correlation))['background'] }
        else:
            return { 'background': 'rgb(255, 255, 255)' }
    return computeTable(variables, TauC, formatCell, colourCell, labels)

In [12]:
def ChiSquaredTable(variables, labels):
    """
    Generate an HTML table of Chi Squared computations for each variable against
    each other variable. This gives a quick visual overview of where there are
    correlations.
    """
    def formatCell(value):
        X2, p, dof, expected = value
        doc, tag, text, line = Doc().ttl()
        with tag('div', klass='value' ):
            text('𝑥²={0:.3g}'.format(X2))
        with tag('div', klass='p-value'):
            text('𝒑<{0:.3g}'.format(p))
        return doc.getvalue()
    def colourCell(value):
        X2, p, dof, expected = value
        if value != None:
            return significanceColour(p)
        else:
            return { 'background': 'rgb(255, 255, 255)' }
    table = computeTable(variables, ChiSquared, formatCell, colourCell, labels)
    return table + significanceKey

In [13]:
def CramersVTable(variables, labels):
    """
    Generate an HTML heat map table of Cramer's V computations for each variable against
    each other variable. This gives a quick visual overview of where there are
    correlations.
    """
    def formatCell(value):
        V, X2, p = value
        doc, tag, text, line = Doc().ttl()
        with tag('div', klass='value'):
            text('{0:.2g}'.format(V))
        with tag('div', klass='p-value'):
            text('(𝑥²={0:.3g} 𝒑<{1:.3g}'.format(X2, p))
        return doc.getvalue()
    def colourCell(value):
        V, X2, p = value
        if value != None:
            return { 'background': heatMapColour(V)['background'] }
        else:
            return { 'background': 'rgb(255, 255, 255)' }
    return computeTable(variables, CramersV, formatCell, colourCell, labels)

In [14]:
def getVariablesAndLabels(variables):
    """
    Take a list of variables where each item may be a tuple or a variable name.
    In the case where there is a tuple, it will be a variable and a label.

    We have groups of variables pertaining to each kind of violence and we often
    want to take the label from the first variable in the group.
    """
    vars = []
    labels = []
    for item in variables:
        if isinstance(item, tuple):
            var, label = item
            vars.append(var)
            labels.append(label)
        else:
            vars.append(item)
            labels.append(NameToLabelMap[item])
    return (vars, labels)


In [27]:

variables, labels = getVariablesAndLabels([
    'Q8_PartnerGotHelp',
    'Q97_PartnerSpecialistHelp',
    'Q9_QualityRecovery',
    'Q10_1_Porn',
    'Q10_2_Media',
    'Q10_3_SexWorkers',
    'Q10_4_CyberSex',
    'Q10_5_PhoneSex',
    'Q10_6_HookUps',
    'Q10_7_Affairs',
    'Q10_8_SexualEntertainment',
    'Q10_9_Voyeurism',
    'Q10_10_Stalking',
    'Q10_11_Exhibitionism',
    'Q10_12_Other',
    ('Q47', 'Slapped or Thrown Something'),
    ('Q55', 'Hit with Fist or Something Else'),
    ('Q59', 'Kicked, Dragged or Beaten'),
    ('Q63', 'Choked or Purposely Burnt'),
    ('Q67', 'Threatened or Used a Weapon'),
    ('Q72', 'Forced Intercourse'),
    ('Q76', "Didn't Want Intercourse, but Afraid"),
    ('Q80', 'Forced to do something Degrading or Humiliating'),
])
table = TauTable(variables, labels)
# print(table)
displayTable(table, "Kendell's Tau-C: SA Behaviours vs IPV")

Husband has Sought Help,Husband has Sought Specialist Help,Quality Recovery,Porn,Other sexualised media,Sex Workers,Cybersex,Phone sex,Hook-ups,Affairs,Sexual entertainment,Voyeurism,Stalking,Exhibitionism,Other behaviours,Slapped or Thrown Something,Hit with Fist or Something Else,"Kicked, Dragged or Beaten",Choked or Purposely Burnt,Threatened or Used a Weapon,Forced Intercourse,"Didn't Want Intercourse, but Afraid",Forced to do something Degrading or Humiliating
Husband has Sought Help,Tau-c=0.39 𝒑<4.31e-57,Tau-c=0.25 𝒑<1.65e-15,Tau-c=0.022 𝒑<0.0801,Tau-c=-0.029 𝒑<0.237,Tau-c=0.0012 𝒑<0.966,Tau-c=-0.026 𝒑<0.345,Tau-c=-0.011 𝒑<0.641,Tau-c=-0.00046 𝒑<0.986,Tau-c=-0.025 𝒑<0.376,Tau-c=-0.057 𝒑<0.0404,Tau-c=-0.0042 𝒑<0.864,Tau-c=-0.012 𝒑<0.351,Tau-c=-0.033 𝒑<0.0697,Tau-c=-0.027 𝒑<0.318,Tau-c=-0.015 𝒑<0.386,Tau-c=-0.017 𝒑<0.101,Tau-c=-0.02 𝒑<0.0394,Tau-c=-0.011 𝒑<0.281,Tau-c=-0.018 𝒑<0.0192,Tau-c=-0.055 𝒑<4.73e-06,Tau-c=-0.035 𝒑<0.038,Tau-c=-0.031 𝒑<0.043
Husband has Sought Specialist Help,Tau-c=0.39 𝒑<4.31e-57,Tau-c=0.48 𝒑<1.21e-31,Tau-c=0.033 𝒑<0.0435,Tau-c=-0.053 𝒑<0.0947,Tau-c=0.064 𝒑<0.0673,Tau-c=0.017 𝒑<0.626,Tau-c=0.023 𝒑<0.47,Tau-c=0.063 𝒑<0.0621,Tau-c=-0.041 𝒑<0.249,Tau-c=-0.056 𝒑<0.12,Tau-c=0.001 𝒑<0.973,Tau-c=-0.0064 𝒑<0.708,Tau-c=-0.015 𝒑<0.532,Tau-c=-0.056 𝒑<0.103,Tau-c=-0.02 𝒑<0.362,Tau-c=-0.022 𝒑<0.102,Tau-c=-0.042 𝒑<0.000864,Tau-c=-0.031 𝒑<0.0188,Tau-c=-0.0093 𝒑<0.358,Tau-c=-0.047 𝒑<0.00204,Tau-c=-0.075 𝒑<0.000539,Tau-c=-0.073 𝒑<0.000201
Quality Recovery,Tau-c=0.25 𝒑<1.65e-15,Tau-c=0.48 𝒑<1.21e-31,Tau-c=0.0055 𝒑<0.795,Tau-c=-0.1 𝒑<0.0138,Tau-c=0.03 𝒑<0.511,Tau-c=-0.075 𝒑<0.0974,Tau-c=-0.061 𝒑<0.133,Tau-c=-0.049 𝒑<0.263,Tau-c=-0.12 𝒑<0.011,Tau-c=-0.036 𝒑<0.44,Tau-c=-0.085 𝒑<0.0369,Tau-c=-0.069 𝒑<0.00187,Tau-c=-0.069 𝒑<0.0225,Tau-c=-0.19 𝒑<1.86e-05,Tau-c=-0.087 𝒑<0.00178,Tau-c=-0.03 𝒑<0.0863,Tau-c=-0.04 𝒑<0.0132,Tau-c=-0.07 𝒑<4.03e-05,Tau-c=-0.04 𝒑<0.00214,Tau-c=-0.072 𝒑<0.000316,Tau-c=-0.12 𝒑<1.49e-05,Tau-c=-0.11 𝒑<2.31e-05
Porn,Tau-c=0.022 𝒑<0.0801,Tau-c=0.033 𝒑<0.0435,Tau-c=0.0055 𝒑<0.795,Tau-c=0.054 𝒑<0.00123,Tau-c=0.025 𝒑<0.171,Tau-c=0.04 𝒑<0.0282,Tau-c=0.045 𝒑<0.00614,Tau-c=0.015 𝒑<0.383,Tau-c=-0.0049 𝒑<0.793,Tau-c=0.065 𝒑<0.000506,Tau-c=0.016 𝒑<0.323,Tau-c=0.0055 𝒑<0.538,Tau-c=0.011 𝒑<0.382,Tau-c=-0.021 𝒑<0.251,Tau-c=0.014 𝒑<0.227,Tau-c=0.00054 𝒑<0.939,Tau-c=0.0066 𝒑<0.319,Tau-c=0.00015 𝒑<0.982,Tau-c=0.0042 𝒑<0.425,Tau-c=-0.019 𝒑<0.0206,Tau-c=0.0069 𝒑<0.547,Tau-c=0.0098 𝒑<0.342
Other sexualised media,Tau-c=-0.029 𝒑<0.237,Tau-c=-0.053 𝒑<0.0947,Tau-c=-0.1 𝒑<0.0138,Tau-c=0.054 𝒑<0.00123,Tau-c=0.014 𝒑<0.694,Tau-c=0.089 𝒑<0.0112,Tau-c=0.085 𝒑<0.00684,Tau-c=0.0047 𝒑<0.89,Tau-c=-0.047 𝒑<0.194,Tau-c=0.13 𝒑<0.000519,Tau-c=0.099 𝒑<0.00158,Tau-c=0.031 𝒑<0.0767,Tau-c=0.074 𝒑<0.00172,Tau-c=0.12 𝒑<0.000395,Tau-c=0.033 𝒑<0.133,Tau-c=0.014 𝒑<0.29,Tau-c=0.0091 𝒑<0.473,Tau-c=0.013 𝒑<0.342,Tau-c=0.0054 𝒑<0.594,Tau-c=0.0038 𝒑<0.809,Tau-c=0.0057 𝒑<0.794,Tau-c=0.022 𝒑<0.271
Sex Workers,Tau-c=0.0012 𝒑<0.966,Tau-c=0.064 𝒑<0.0673,Tau-c=0.03 𝒑<0.511,Tau-c=0.025 𝒑<0.171,Tau-c=0.014 𝒑<0.694,Tau-c=0.16 𝒑<3.92e-05,Tau-c=0.16 𝒑<2.27e-06,Tau-c=0.28 𝒑<2.34e-14,Tau-c=0.21 𝒑<1.74e-07,Tau-c=0.35 𝒑<4.34e-19,Tau-c=-0.023 𝒑<0.51,Tau-c=0.025 𝒑<0.192,Tau-c=0.074 𝒑<0.00446,Tau-c=-0.012 𝒑<0.75,Tau-c=0.065 𝒑<0.00616,Tau-c=0.029 𝒑<0.0556,Tau-c=0.029 𝒑<0.0362,Tau-c=0.0098 𝒑<0.506,Tau-c=0.023 𝒑<0.044,Tau-c=-0.0078 𝒑<0.648,Tau-c=-0.045 𝒑<0.0641,Tau-c=-0.031 𝒑<0.15
Cybersex,Tau-c=-0.026 𝒑<0.345,Tau-c=0.017 𝒑<0.626,Tau-c=-0.075 𝒑<0.0974,Tau-c=0.04 𝒑<0.0282,Tau-c=0.089 𝒑<0.0112,Tau-c=0.16 𝒑<3.92e-05,Tau-c=0.36 𝒑<2.41e-25,Tau-c=0.36 𝒑<7.21e-22,Tau-c=0.26 𝒑<7.81e-11,Tau-c=0.15 𝒑<0.000207,Tau-c=0.081 𝒑<0.0191,Tau-c=0.046 𝒑<0.0172,Tau-c=0.12 𝒑<9.5e-06,Tau-c=0.048 𝒑<0.215,Tau-c=0.014 𝒑<0.562,Tau-c=-0.00044 𝒑<0.977,Tau-c=0.0072 𝒑<0.607,Tau-c=0.016 𝒑<0.263,Tau-c=0.022 𝒑<0.0473,Tau-c=-0.016 𝒑<0.36,Tau-c=-0.017 𝒑<0.473,Tau-c=-0.011 𝒑<0.614
Phone sex,Tau-c=-0.011 𝒑<0.641,Tau-c=0.023 𝒑<0.47,Tau-c=-0.061 𝒑<0.133,Tau-c=0.045 𝒑<0.00614,Tau-c=0.085 𝒑<0.00684,Tau-c=0.16 𝒑<2.

In [15]:
variables, labels = getVariablesAndLabels([
    'Q10_1_Porn',
    'Q10_2_Media',
    'Q10_3_SexWorkers',
    'Q10_4_CyberSex',
    'Q10_5_PhoneSex',
    'Q10_6_HookUps',
    'Q10_7_Affairs',
    'Q10_8_SexualEntertainment',
    'Q10_9_Voyeurism',
    'Q10_10_Stalking',
    'Q10_11_Exhibitionism',
    'Q10_12_Other',
    ('Q47', 'Slapped or Thrown Something'),
    ('Q55', 'Hit with Fist or Something Else'),
    ('Q59', 'Kicked, Dragged or Beaten'),
    ('Q63', 'Choked or Purposely Burnt'),
    ('Q67', 'Threatened or Used a Weapon'),
    ('Q72', 'Forced Intercourse'),
    ('Q76', "Didn't Want Intercourse, but Afraid"),
    ('Q80', 'Forced to do something Degrading or Humiliating'),
    ('Q85', 'Threatened harm to children'),
    ('Q89', 'Physical violence against children'),
])
table = ChiSquaredTable(variables, labels)
writeTable(table, 'ChiSquared_ActingOutBehviours_vs_current_IPV.html')
displayTable(table, 'Chi Squared: Acting Out Behaviours vs current IPV')

Porn,Other sexualised media,Sex Workers,Cybersex,Phone sex,Hook-ups,Affairs,Sexual entertainment,Voyeurism,Stalking,Exhibitionism,Other behaviours,Slapped or Thrown Something,Hit with Fist or Something Else,"Kicked, Dragged or Beaten",Choked or Purposely Burnt,Threatened or Used a Weapon,Forced Intercourse,"Didn't Want Intercourse, but Afraid",Forced to do something Degrading or Humiliating,Threatened harm to children,Physical violence against children
Porn,𝑥²=10.5 𝒑<0.00121,𝑥²=1.88 𝒑<0.17,𝑥²=4.82 𝒑<0.0281,𝑥²=7.52 𝒑<0.00609,𝑥²=0.761 𝒑<0.383,𝑥²=0.0689 𝒑<0.793,𝑥²=12.1 𝒑<0.0005,𝑥²=0.979 𝒑<0.322,𝑥²=0.379 𝒑<0.538,𝑥²=0.765 𝒑<0.382,𝑥²=1.32 𝒑<0.251,𝑥²=1.46 𝒑<0.227,𝑥²=0.00578 𝒑<0.939,𝑥²=0.996 𝒑<0.318,𝑥²=0.000495 𝒑<0.982,𝑥²=0.638 𝒑<0.425,𝑥²=5.37 𝒑<0.0205,𝑥²=0.363 𝒑<0.547,𝑥²=0.904 𝒑<0.342,𝑥²=0.00638 𝒑<0.936,𝑥²=1.39 𝒑<0.238
Other sexualised media,𝑥²=10.5 𝒑<0.00121,𝑥²=0.155 𝒑<0.693,𝑥²=6.45 𝒑<0.0111,𝑥²=7.33 𝒑<0.00679,𝑥²=0.0192 𝒑<0.89,𝑥²=1.69 𝒑<0.194,𝑥²=12.1 𝒑<0.000513,𝑥²=10 𝒑<0.00156,𝑥²=3.14 𝒑<0.0765,𝑥²=9.84 𝒑<0.0017,𝑥²=12.6 𝒑<0.00039,𝑥²=2.26 𝒑<0.133,𝑥²=1.12 𝒑<0.289,𝑥²=0.517 𝒑<0.472,𝑥²=0.905 𝒑<0.341,𝑥²=0.285 𝒑<0.594,𝑥²=0.0588 𝒑<0.808,𝑥²=0.0686 𝒑<0.793,𝑥²=1.21 𝒑<0.271,𝑥²=0.274 𝒑<0.6,𝑥²=0.82 𝒑<0.365
Sex Workers,𝑥²=1.88 𝒑<0.17,𝑥²=0.155 𝒑<0.693,𝑥²=16.9 𝒑<3.85e-05,𝑥²=22.4 𝒑<2.23e-06,𝑥²=58.3 𝒑<2.22e-14,𝑥²=27.3 𝒑<1.7e-07,𝑥²=79.9 𝒑<4.04e-19,𝑥²=0.434 𝒑<0.51,𝑥²=1.7 𝒑<0.192,𝑥²=8.1 𝒑<0.00442,𝑥²=0.102 𝒑<0.75,𝑥²=7.52 𝒑<0.00612,𝑥²=3.67 𝒑<0.0554,𝑥²=4.4 𝒑<0.036,𝑥²=0.443 𝒑<0.505,𝑥²=4.06 𝒑<0.0438,𝑥²=0.209 𝒑<0.647,𝑥²=3.43 𝒑<0.0639,𝑥²=2.08 𝒑<0.149,𝑥²=0.823 𝒑<0.364,𝑥²=0.662 𝒑<0.416
Cybersex,𝑥²=4.82 𝒑<0.0281,𝑥²=6.45 𝒑<0.0111,𝑥²=16.9 𝒑<3.85e-05,𝑥²=108 𝒑<2.19e-25,𝑥²=92.5 𝒑<6.64e-22,𝑥²=42.4 𝒑<7.51e-11,𝑥²=13.8 𝒑<0.000205,𝑥²=5.5 𝒑<0.019,𝑥²=5.68 𝒑<0.0171,𝑥²=19.6 𝒑<9.33e-06,𝑥²=1.54 𝒑<0.214,𝑥²=0.336 𝒑<0.562,𝑥²=0.000843 𝒑<0.977,𝑥²=0.265 𝒑<0.607,𝑥²=1.25 𝒑<0.263,𝑥²=3.94 𝒑<0.0471,𝑥²=0.839 𝒑<0.36,𝑥²=0.516 𝒑<0.472,𝑥²=0.254 𝒑<0.614,𝑥²=0.196 𝒑<0.658,𝑥²=2.28 𝒑<0.131
Phone sex,𝑥²=7.52 𝒑<0.00609,𝑥²=7.33 𝒑<0.00679,𝑥²=22.4 𝒑<2.23e-06,𝑥²=108 𝒑<2.19e-25,𝑥²=30.9 𝒑<2.76e-08,𝑥²=11.1 𝒑<0.000852,𝑥²=24.5 𝒑<7.42e-07,𝑥²=4.65 𝒑<0.0311,𝑥²=2.83 𝒑<0.0924,𝑥²=3.74 𝒑<0.0532,𝑥²=1.33 𝒑<0.249,𝑥²=2.72 𝒑<0.0989,𝑥²=0.199 𝒑<0.656,𝑥²=0.00422 𝒑<0.948,𝑥²=0.585 𝒑<0.444,𝑥²=5.64 𝒑<0.0176,𝑥²=6.16 𝒑<0.0131,𝑥²=0.585 𝒑<0.444,𝑥²=0.056 𝒑<0.813,𝑥²=1.83 𝒑<0.176,𝑥²=0.921 𝒑<0.337
Hook-ups,𝑥²=0.761 𝒑<0.383,𝑥²=0.0192 𝒑<0.89,𝑥²=58.3 𝒑<2.22e-14,𝑥²=92.5 𝒑<6.64e-22,𝑥²=30.9 𝒑<2.76e-08,𝑥²=74.7 𝒑<5.4e-18,𝑥²=28.5 𝒑<9.57e-08,𝑥²=0.000768 𝒑<0.978,𝑥²=0.576 𝒑<0.448,𝑥²=10.3 𝒑<0.00133,𝑥²=0.113 𝒑<0.737,𝑥²=2.98 𝒑<0.0843,𝑥²=0.00213 𝒑<0.963,𝑥²=0.193 𝒑<0.66,𝑥²=2.65 𝒑<0.103,𝑥²=3.07 𝒑<0.0797,𝑥²=0.703 𝒑<0.402,𝑥²=1.34 𝒑<0.246,𝑥²=1.35 𝒑<0.245,𝑥²=0.627 𝒑<0.428,𝑥²=0.214 𝒑<0.644
Affairs,𝑥²=0.0689 𝒑<0.793,𝑥²=1.69 𝒑<0.194,𝑥²=27.3 𝒑<1.7e-07,𝑥²=42.4 𝒑<7.51e-11,𝑥²=11.1 𝒑<0.000852,𝑥²=74.7 𝒑<5.4e-18,𝑥²=11.9 𝒑<0.000562,𝑥²=0.853 𝒑<0.356,𝑥²=2.64 𝒑<0.104,𝑥²=16.9 𝒑<3.94e-05,𝑥²=0.113 𝒑<0.737,𝑥²=0.683 𝒑<0.409,𝑥²=0.00926 𝒑<0.923,𝑥²=1.03 𝒑<0.311,𝑥²=6.09 𝒑<0.0136,𝑥²=7.71 𝒑<0.00549,𝑥²=3.13 𝒑<0.0771,𝑥²=0.17 𝒑<0.68,𝑥²=0.883 𝒑<0.347,𝑥²=0.00959 𝒑<0.922,𝑥²=0.277 𝒑<0.599
Sexual entertainment,𝑥²=12.1 𝒑<0.0005,𝑥²=12.1 𝒑<0.000513,𝑥²=79.9 𝒑<4.04e-19,𝑥²=13.8 𝒑<0.000205,𝑥²=24.5 𝒑<7.42e-07,𝑥²=28.5 𝒑<9.57e-08,𝑥²=11.9 𝒑<0.000562,𝑥²=3.71 𝒑<0.0539,𝑥²=4.34 𝒑<0.0373,𝑥²=6.94 𝒑<0.00845,𝑥²=0.16 𝒑<0.689,𝑥²=4.45 𝒑<0.0349,𝑥²=3.34 𝒑<0.0677,𝑥²=2.49 𝒑<0.114,𝑥²=0.406 𝒑<0.524,𝑥²=2.53 𝒑<0.111,𝑥²=0.0447 𝒑<0.833,𝑥²=0.0567 𝒑<0.812,𝑥²=1.09 𝒑<0.297,𝑥²=2.01 𝒑<0.157,𝑥²=0.249 𝒑<0.618
Voyeurism,𝑥²=0.979 𝒑<0.322,𝑥²=10 𝒑<0.00156,𝑥²=0.434 𝒑<0.51,𝑥²=5.5 𝒑<0.019,𝑥²=4.65 𝒑<0.0311,𝑥²=0.000768 𝒑<0.978,𝑥²=0.853 𝒑<0.356,𝑥²=3.71 𝒑<0.0539,𝑥²=39.6 𝒑<3.12e-10,𝑥²=65.5 𝒑<5.69e-16,𝑥²=33.1 𝒑<8.68e-09,𝑥²=0.419 𝒑<0.518,𝑥²=0.199 𝒑<0.656,𝑥²=1.48 𝒑<0.224,𝑥²=0.0483 𝒑<0.826,𝑥²=5.64 𝒑<0.0176,𝑥²=0.0185 𝒑<0.892,𝑥²=4.93 𝒑<0.0264,𝑥²=2.55 𝒑<0.11,𝑥²=1.39 𝒑<0.238,𝑥²=0.0197 𝒑<0.888
Stalking,𝑥²=0.379 𝒑<0.538,𝑥²=3.14 𝒑<0.0765,𝑥²=1.7 𝒑<0.192,𝑥²=5.68 𝒑<0.0171,𝑥²=2.83 𝒑<0.0924,𝑥²=0.576 𝒑<0.448,𝑥²=2.64 𝒑<0.104,𝑥²=4.34 𝒑<0.0373,𝑥²=39.6 𝒑<3.12e-10,𝑥²

In [43]:
variables, labels = getVariablesAndLabels([
    'Q10_1_Porn',
    'Q10_2_Media',
    'Q10_3_SexWorkers',
    'Q10_4_CyberSex',
    'Q10_5_PhoneSex',
    'Q10_6_HookUps',
    'Q10_7_Affairs',
    'Q10_8_SexualEntertainment',
    'Q10_9_Voyeurism',
    'Q10_10_Stalking',
    'Q10_11_Exhibitionism',
    'Q10_12_Other',
    ('Q46_PA_SlappedThrown', 'Slapped or Thrown Something'),
    ('Q54_PA_HitFistObject', 'Hit with Fist or Something Else'),
    ('Q58_PA_KickedDraggedBeaten', 'Kicked, Dragged or Beaten'),
    ('Q62_PA_ChockedStrangledBurnt', 'Choked or Purposely Burnt'),
    ('Q66_PA_Weapon', 'Threatened or Used a Weapon'),
    ('Q71_SA_ForcedIntercourse', 'Forced Intercourse'),
    ('Q75_SA_CapitulatedFear', "Didn't Want Intercourse, but Afraid"),
    ('Q79_SA_Degrading', 'Forced to do something Degrading or Humiliating'),
    ('Q84_CA_ThreatenHarm', 'Threatened harm to children'),
    ('Q88_CA_PhysicalViolence', 'Physical violence against children'),
])
table = ChiSquaredTable(variables, labels)
writeTable(table, 'ChiSquared_ActingOutBehviours_vs_IPV.html')
displayTable(table, 'Chi Squared: Acting Out Behaviours vs IPV')

Porn,Other sexualised media,Sex Workers,Cybersex,Phone sex,Hook-ups,Affairs,Sexual entertainment,Voyeurism,Stalking,Exhibitionism,Other behaviours,Slapped or Thrown Something,Hit with Fist or Something Else,"Kicked, Dragged or Beaten",Choked or Purposely Burnt,Threatened or Used a Weapon,Forced Intercourse,"Didn't Want Intercourse, but Afraid",Forced to do something Degrading or Humiliating,Threatened harm to children,Physical violence against children
Porn,𝑥²=10.5 𝒑<0.00121,𝑥²=1.88 𝒑<0.17,𝑥²=4.82 𝒑<0.0281,𝑥²=7.52 𝒑<0.00609,𝑥²=0.761 𝒑<0.383,𝑥²=0.0689 𝒑<0.793,𝑥²=12.1 𝒑<0.0005,𝑥²=0.979 𝒑<0.322,𝑥²=0.379 𝒑<0.538,𝑥²=0.765 𝒑<0.382,𝑥²=1.32 𝒑<0.251,𝑥²=0.301 𝒑<0.583,𝑥²=0.328 𝒑<0.567,𝑥²=0.298 𝒑<0.585,𝑥²=0.0836 𝒑<0.773,𝑥²=0.00883 𝒑<0.925,𝑥²=0.0279 𝒑<0.867,𝑥²=0.0572 𝒑<0.811,𝑥²=0.295 𝒑<0.587,𝑥²=2.23 𝒑<0.136,𝑥²=0.0303 𝒑<0.862
Other sexualised media,𝑥²=10.5 𝒑<0.00121,𝑥²=0.155 𝒑<0.693,𝑥²=6.45 𝒑<0.0111,𝑥²=7.33 𝒑<0.00679,𝑥²=0.0192 𝒑<0.89,𝑥²=1.69 𝒑<0.194,𝑥²=12.1 𝒑<0.000513,𝑥²=10 𝒑<0.00156,𝑥²=3.14 𝒑<0.0765,𝑥²=9.84 𝒑<0.0017,𝑥²=12.6 𝒑<0.00039,𝑥²=1.37 𝒑<0.241,𝑥²=0.411 𝒑<0.521,𝑥²=1.4 𝒑<0.236,𝑥²=3.6 𝒑<0.0578,𝑥²=0.517 𝒑<0.472,𝑥²=1.69 𝒑<0.194,𝑥²=1.55 𝒑<0.214,𝑥²=7.59 𝒑<0.00586,𝑥²=1.86 𝒑<0.172,𝑥²=1.37 𝒑<0.242
Sex Workers,𝑥²=1.88 𝒑<0.17,𝑥²=0.155 𝒑<0.693,𝑥²=16.9 𝒑<3.85e-05,𝑥²=22.4 𝒑<2.23e-06,𝑥²=58.3 𝒑<2.22e-14,𝑥²=27.3 𝒑<1.7e-07,𝑥²=79.9 𝒑<4.04e-19,𝑥²=0.434 𝒑<0.51,𝑥²=1.7 𝒑<0.192,𝑥²=8.1 𝒑<0.00442,𝑥²=0.102 𝒑<0.75,𝑥²=5.63 𝒑<0.0176,𝑥²=3.73 𝒑<0.0536,𝑥²=2.61 𝒑<0.106,𝑥²=0.55 𝒑<0.458,𝑥²=2.5 𝒑<0.114,𝑥²=0.775 𝒑<0.379,𝑥²=1.39 𝒑<0.239,𝑥²=0.44 𝒑<0.507,𝑥²=0.186 𝒑<0.666,𝑥²=0.352 𝒑<0.553
Cybersex,𝑥²=4.82 𝒑<0.0281,𝑥²=6.45 𝒑<0.0111,𝑥²=16.9 𝒑<3.85e-05,𝑥²=108 𝒑<2.19e-25,𝑥²=92.5 𝒑<6.64e-22,𝑥²=42.4 𝒑<7.51e-11,𝑥²=13.8 𝒑<0.000205,𝑥²=5.5 𝒑<0.019,𝑥²=5.68 𝒑<0.0171,𝑥²=19.6 𝒑<9.33e-06,𝑥²=1.54 𝒑<0.214,𝑥²=4.24 𝒑<0.0396,𝑥²=3.52 𝒑<0.0607,𝑥²=3.82 𝒑<0.0506,𝑥²=10.8 𝒑<0.001,𝑥²=6.64 𝒑<0.00999,𝑥²=0.00484 𝒑<0.945,𝑥²=0.992 𝒑<0.319,𝑥²=0.82 𝒑<0.365,𝑥²=1.42 𝒑<0.233,𝑥²=2.05 𝒑<0.153
Phone sex,𝑥²=7.52 𝒑<0.00609,𝑥²=7.33 𝒑<0.00679,𝑥²=22.4 𝒑<2.23e-06,𝑥²=108 𝒑<2.19e-25,𝑥²=30.9 𝒑<2.76e-08,𝑥²=11.1 𝒑<0.000852,𝑥²=24.5 𝒑<7.42e-07,𝑥²=4.65 𝒑<0.0311,𝑥²=2.83 𝒑<0.0924,𝑥²=3.74 𝒑<0.0532,𝑥²=1.33 𝒑<0.249,𝑥²=10.4 𝒑<0.00128,𝑥²=0.621 𝒑<0.431,𝑥²=1.16 𝒑<0.281,𝑥²=4.92 𝒑<0.0265,𝑥²=2.76 𝒑<0.0968,𝑥²=1.22 𝒑<0.27,𝑥²=0.321 𝒑<0.571,𝑥²=0.00406 𝒑<0.949,𝑥²=7.3e-05 𝒑<0.993,𝑥²=0.218 𝒑<0.641
Hook-ups,𝑥²=0.761 𝒑<0.383,𝑥²=0.0192 𝒑<0.89,𝑥²=58.3 𝒑<2.22e-14,𝑥²=92.5 𝒑<6.64e-22,𝑥²=30.9 𝒑<2.76e-08,𝑥²=74.7 𝒑<5.4e-18,𝑥²=28.5 𝒑<9.57e-08,𝑥²=0.000768 𝒑<0.978,𝑥²=0.576 𝒑<0.448,𝑥²=10.3 𝒑<0.00133,𝑥²=0.113 𝒑<0.737,𝑥²=8.69 𝒑<0.0032,𝑥²=1.53 𝒑<0.216,𝑥²=3.35 𝒑<0.0674,𝑥²=9.85 𝒑<0.0017,𝑥²=4.18 𝒑<0.0409,𝑥²=0.785 𝒑<0.376,𝑥²=1.76 𝒑<0.185,𝑥²=1.94 𝒑<0.164,𝑥²=2.81 𝒑<0.0938,𝑥²=0.487 𝒑<0.485
Affairs,𝑥²=0.0689 𝒑<0.793,𝑥²=1.69 𝒑<0.194,𝑥²=27.3 𝒑<1.7e-07,𝑥²=42.4 𝒑<7.51e-11,𝑥²=11.1 𝒑<0.000852,𝑥²=74.7 𝒑<5.4e-18,𝑥²=11.9 𝒑<0.000562,𝑥²=0.853 𝒑<0.356,𝑥²=2.64 𝒑<0.104,𝑥²=16.9 𝒑<3.94e-05,𝑥²=0.113 𝒑<0.737,𝑥²=3.51 𝒑<0.061,𝑥²=1.9 𝒑<0.169,𝑥²=3.93 𝒑<0.0474,𝑥²=5.67 𝒑<0.0173,𝑥²=12.4 𝒑<0.000437,𝑥²=5.97 𝒑<0.0146,𝑥²=0.924 𝒑<0.337,𝑥²=4.55 𝒑<0.0329,𝑥²=2.81 𝒑<0.0939,𝑥²=0.0598 𝒑<0.807
Sexual entertainment,𝑥²=12.1 𝒑<0.0005,𝑥²=12.1 𝒑<0.000513,𝑥²=79.9 𝒑<4.04e-19,𝑥²=13.8 𝒑<0.000205,𝑥²=24.5 𝒑<7.42e-07,𝑥²=28.5 𝒑<9.57e-08,𝑥²=11.9 𝒑<0.000562,𝑥²=3.71 𝒑<0.0539,𝑥²=4.34 𝒑<0.0373,𝑥²=6.94 𝒑<0.00845,𝑥²=0.16 𝒑<0.689,𝑥²=9.74 𝒑<0.0018,𝑥²=3.27 𝒑<0.0705,𝑥²=4.29 𝒑<0.0383,𝑥²=1.49 𝒑<0.222,𝑥²=0.331 𝒑<0.565,𝑥²=1.98 𝒑<0.159,𝑥²=0.408 𝒑<0.523,𝑥²=4.11 𝒑<0.0426,𝑥²=2.92 𝒑<0.0872,𝑥²=1.95 𝒑<0.163
Voyeurism,𝑥²=0.979 𝒑<0.322,𝑥²=10 𝒑<0.00156,𝑥²=0.434 𝒑<0.51,𝑥²=5.5 𝒑<0.019,𝑥²=4.65 𝒑<0.0311,𝑥²=0.000768 𝒑<0.978,𝑥²=0.853 𝒑<0.356,𝑥²=3.71 𝒑<0.0539,𝑥²=39.6 𝒑<3.12e-10,𝑥²=65.5 𝒑<5.69e-16,𝑥²=33.1 𝒑<8.68e-09,𝑥²=1.04 𝒑<0.307,𝑥²=0.000933 𝒑<0.976,𝑥²=0.419 𝒑<0.517,𝑥²=1.28 𝒑<0.258,𝑥²=7.9 𝒑<0.00494,𝑥²=1.16 𝒑<0.282,𝑥²=11.1 𝒑<0.000848,𝑥²=17.8 𝒑<2.44e-05,𝑥²=0.614 𝒑<0.433,𝑥²=0.633 𝒑<0.426
Stalking,𝑥²=0.379 𝒑<0.538,𝑥²=3.14 𝒑<0.0765,𝑥²=1.7 𝒑<0.192,𝑥²=5.68 𝒑<0.0171,𝑥²=2.83 𝒑<0.0924,𝑥²=0.576 𝒑<0.448,𝑥²=2.64 𝒑<0.104,𝑥²=4.34 𝒑<0.0373,𝑥²=39.6 𝒑<3.12e-10,𝑥²=32.3 

In [38]:
variables, labels = getVariablesAndLabels([
    'Q10_1_Porn',
    'Q10_2_Media',
    'Q10_3_SexWorkers',
    'Q10_4_CyberSex',
    'Q10_5_PhoneSex',
    'Q10_6_HookUps',
    'Q10_7_Affairs',
    'Q10_8_SexualEntertainment',
    'Q10_9_Voyeurism',
    'Q10_10_Stalking',
    'Q10_11_Exhibitionism',
    'Q10_12_Other',
    ('Q47', 'Slapped or Thrown Something'),
    ('Q55', 'Hit with Fist or Something Else'),
    ('Q59', 'Kicked, Dragged or Beaten'),
    ('Q63', 'Choked or Purposely Burnt'),
    ('Q67', 'Threatened or Used a Weapon'),
    ('Q72', 'Forced Intercourse'),
    ('Q76', "Didn't Want Intercourse, but Afraid"),
    ('Q80', 'Forced to do something Degrading or Humiliating'),
])
table = CramersVTable(variables, labels)
writeTable(table, 'CramersV_ActingOutBehviours_vs_IPV.html')
displayTable(table, 'Cramers V: Acting Out Behaviours vs IPV')

Porn,Other sexualised media,Sex Workers,Cybersex,Phone sex,Hook-ups,Affairs,Sexual entertainment,Voyeurism,Stalking,Exhibitionism,Other behaviours,Slapped or Thrown Something,Hit with Fist or Something Else,"Kicked, Dragged or Beaten",Choked or Purposely Burnt,Threatened or Used a Weapon,Forced Intercourse,"Didn't Want Intercourse, but Afraid",Forced to do something Degrading or Humiliating
Porn,0.14 (𝑥²=10.5 𝒑<0.00121,0.058 (𝑥²=1.88 𝒑<0.17,0.093 (𝑥²=4.82 𝒑<0.0281,0.12 (𝑥²=7.52 𝒑<0.00609,0.037 (𝑥²=0.761 𝒑<0.383,0.011 (𝑥²=0.0689 𝒑<0.793,0.15 (𝑥²=12.1 𝒑<0.0005,0.042 (𝑥²=0.979 𝒑<0.322,0.026 (𝑥²=0.379 𝒑<0.538,0.037 (𝑥²=0.765 𝒑<0.382,0.049 (𝑥²=1.32 𝒑<0.251,0.051 (𝑥²=1.46 𝒑<0.227,0.0032 (𝑥²=0.00578 𝒑<0.939,0.042 (𝑥²=0.996 𝒑<0.318,0.00094 (𝑥²=0.000495 𝒑<0.982,0.034 (𝑥²=0.638 𝒑<0.425,0.098 (𝑥²=5.37 𝒑<0.0205,0.026 (𝑥²=0.363 𝒑<0.547,0.04 (𝑥²=0.904 𝒑<0.342
Other sexualised media,0.14 (𝑥²=10.5 𝒑<0.00121,0.017 (𝑥²=0.155 𝒑<0.693,0.11 (𝑥²=6.45 𝒑<0.0111,0.11 (𝑥²=7.33 𝒑<0.00679,0.0059 (𝑥²=0.0192 𝒑<0.89,0.055 (𝑥²=1.69 𝒑<0.194,0.15 (𝑥²=12.1 𝒑<0.000513,0.13 (𝑥²=10 𝒑<0.00156,0.075 (𝑥²=3.14 𝒑<0.0765,0.13 (𝑥²=9.84 𝒑<0.0017,0.15 (𝑥²=12.6 𝒑<0.00039,0.064 (𝑥²=2.26 𝒑<0.133,0.045 (𝑥²=1.12 𝒑<0.289,0.03 (𝑥²=0.517 𝒑<0.472,0.04 (𝑥²=0.905 𝒑<0.341,0.023 (𝑥²=0.285 𝒑<0.594,0.01 (𝑥²=0.0588 𝒑<0.808,0.011 (𝑥²=0.0686 𝒑<0.793,0.047 (𝑥²=1.21 𝒑<0.271
Sex Workers,0.058 (𝑥²=1.88 𝒑<0.17,0.017 (𝑥²=0.155 𝒑<0.693,0.17 (𝑥²=16.9 𝒑<3.85e-05,0.2 (𝑥²=22.4 𝒑<2.23e-06,0.32 (𝑥²=58.3 𝒑<2.22e-14,0.22 (𝑥²=27.3 𝒑<1.7e-07,0.38 (𝑥²=79.9 𝒑<4.04e-19,0.028 (𝑥²=0.434 𝒑<0.51,0.055 (𝑥²=1.7 𝒑<0.192,0.12 (𝑥²=8.1 𝒑<0.00442,0.014 (𝑥²=0.102 𝒑<0.75,0.12 (𝑥²=7.52 𝒑<0.00612,0.081 (𝑥²=3.67 𝒑<0.0554,0.089 (𝑥²=4.4 𝒑<0.036,0.028 (𝑥²=0.443 𝒑<0.505,0.085 (𝑥²=4.06 𝒑<0.0438,0.019 (𝑥²=0.209 𝒑<0.647,0.078 (𝑥²=3.43 𝒑<0.0639,0.061 (𝑥²=2.08 𝒑<0.149
Cybersex,0.093 (𝑥²=4.82 𝒑<0.0281,0.11 (𝑥²=6.45 𝒑<0.0111,0.17 (𝑥²=16.9 𝒑<3.85e-05,0.44 (𝑥²=108 𝒑<2.19e-25,0.41 (𝑥²=92.5 𝒑<6.64e-22,0.28 (𝑥²=42.4 𝒑<7.51e-11,0.16 (𝑥²=13.8 𝒑<0.000205,0.099 (𝑥²=5.5 𝒑<0.019,0.1 (𝑥²=5.68 𝒑<0.0171,0.19 (𝑥²=19.6 𝒑<9.33e-06,0.053 (𝑥²=1.54 𝒑<0.214,0.025 (𝑥²=0.336 𝒑<0.562,0.0012 (𝑥²=0.000843 𝒑<0.977,0.022 (𝑥²=0.265 𝒑<0.607,0.047 (𝑥²=1.25 𝒑<0.263,0.084 (𝑥²=3.94 𝒑<0.0471,0.039 (𝑥²=0.839 𝒑<0.36,0.03 (𝑥²=0.516 𝒑<0.472,0.021 (𝑥²=0.254 𝒑<0.614
Phone sex,0.12 (𝑥²=7.52 𝒑<0.00609,0.11 (𝑥²=7.33 𝒑<0.00679,0.2 (𝑥²=22.4 𝒑<2.23e-06,0.44 (𝑥²=108 𝒑<2.19e-25,0.24 (𝑥²=30.9 𝒑<2.76e-08,0.14 (𝑥²=11.1 𝒑<0.000852,0.21 (𝑥²=24.5 𝒑<7.42e-07,0.091 (𝑥²=4.65 𝒑<0.0311,0.071 (𝑥²=2.83 𝒑<0.0924,0.082 (𝑥²=3.74 𝒑<0.0532,0.049 (𝑥²=1.33 𝒑<0.249,0.07 (𝑥²=2.72 𝒑<0.0989,0.019 (𝑥²=0.199 𝒑<0.656,0.0027 (𝑥²=0.00422 𝒑<0.948,0.032 (𝑥²=0.585 𝒑<0.444,0.1 (𝑥²=5.64 𝒑<0.0176,0.11 (𝑥²=6.16 𝒑<0.0131,0.032 (𝑥²=0.585 𝒑<0.444,0.01 (𝑥²=0.056 𝒑<0.813
Hook-ups,0.037 (𝑥²=0.761 𝒑<0.383,0.0059 (𝑥²=0.0192 𝒑<0.89,0.32 (𝑥²=58.3 𝒑<2.22e-14,0.41 (𝑥²=92.5 𝒑<6.64e-22,0.24 (𝑥²=30.9 𝒑<2.76e-08,0.37 (𝑥²=74.7 𝒑<5.4e-18,0.23 (𝑥²=28.5 𝒑<9.57e-08,0.0012 (𝑥²=0.000768 𝒑<0.978,0.032 (𝑥²=0.576 𝒑<0.448,0.14 (𝑥²=10.3 𝒑<0.00133,0.014 (𝑥²=0.113 𝒑<0.737,0.073 (𝑥²=2.98 𝒑<0.0843,0.002 (𝑥²=0.00213 𝒑<0.963,0.019 (𝑥²=0.193 𝒑<0.66,0.069 (𝑥²=2.65 𝒑<0.103,0.074 (𝑥²=3.07 𝒑<0.0797,0.035 (𝑥²=0.703 𝒑<0.402,0.049 (𝑥²=1.34 𝒑<0.246,0.049 (𝑥²=1.35 𝒑<0.245
Affairs,0.011 (𝑥²=0.0689 𝒑<0.793,0.055 (𝑥²=1.69 𝒑<0.194,0.22 (𝑥²=27.3 𝒑<1.7e-07,0.28 (𝑥²=42.4 𝒑<7.51e-11,0.14 (𝑥²=11.1 𝒑<0.000852,0.37 (𝑥²=74.7 𝒑<5.4e-18,0.15 (𝑥²=11.9 𝒑<0.000562,0.039 (𝑥²=0.853 𝒑<0.356,0.069 (𝑥²=2.64 𝒑<0.104,0.17 (𝑥²=16.9 𝒑<3.94e-05,0.014 (𝑥²=0.113 𝒑<0.737,0.035 (𝑥²=0.683 𝒑<0.409,0.0041 (𝑥²=0.00926 𝒑<0.923,0.043 (𝑥²=1.03 𝒑<0.311,0.1 (𝑥²=6.09 𝒑<0.0136,0.12 (𝑥²=7.71 𝒑<0.00549,0.075 (𝑥²=3.13 𝒑<0.0771,0.017 (𝑥²=0.17 𝒑<0.68,0.04 (𝑥²=0.883 𝒑<0.347
Sexual entertainment,0.15 (𝑥²=12.1 𝒑<0.0005,0.15 (𝑥²=12.1 𝒑<0.000513,0.38 (𝑥²=79.9 𝒑<4.04e-19,0.16 (𝑥²=13.8 𝒑<0.000205,0.21 (𝑥²=24.5 𝒑<7.42e-07,0.23 (𝑥²=28.5 𝒑<9.57e-08,0.15 (𝑥²=11.9 𝒑<0.000562,0.082 (𝑥²=3.71 𝒑<0.0539,0.088 (𝑥²=4.34 𝒑<0.0373,0.11 (𝑥²=6.94 𝒑<0.00845,0.017 (𝑥²=0.16 𝒑<0.689,0.089 (𝑥²=4.45 𝒑<0.0349,0.077 (𝑥²=3.34 𝒑<0.0677,0.067

In [39]:
variables, labels = getVariablesAndLabels([
    'Q8_PartnerGotHelp',
    'Q97_PartnerSpecialistHelp',
    'Q9_QualityRecovery',
    ('Q47', 'Slapped or Thrown Something'),
    ('Q55', 'Hit with Fist or Something Else'),
    ('Q59', 'Kicked, Dragged or Beaten'),
    ('Q63', 'Choked or Purposely Burnt'),
    ('Q67', 'Threatened or Used a Weapon'),
    ('Q72', 'Forced Intercourse'),
    ('Q76', "Didn't Want Intercourse, but Afraid"),
    ('Q80', 'Forced to do something Degrading or Humiliating'),
])
table = CramersVTable(variables, labels)
# print(table)
writeTable(table, 'CramersV_RecoveryBehviours_vs_IPV.html')
displayTable(table, 'Cramers V: Recovery Behaviours vs IPV')

Husband has Sought Help,Husband has Sought Specialist Help,Quality Recovery,Slapped or Thrown Something,Hit with Fist or Something Else,"Kicked, Dragged or Beaten",Choked or Purposely Burnt,Threatened or Used a Weapon,Forced Intercourse,"Didn't Want Intercourse, but Afraid",Forced to do something Degrading or Humiliating
Husband has Sought Help,0.67 (𝑥²=254 𝒑<3.43e-57,0.19 (𝑥²=78.6 𝒑<3.5e-16,0.037 (𝑥²=0.753 𝒑<0.385,0.07 (𝑥²=2.7 𝒑<0.1,0.087 (𝑥²=4.25 𝒑<0.0392,0.046 (𝑥²=1.16 𝒑<0.281,0.099 (𝑥²=5.5 𝒑<0.0191,0.19 (𝑥²=21 𝒑<4.64e-06,0.088 (𝑥²=4.31 𝒑<0.0378,0.086 (𝑥²=4.1 𝒑<0.0428
Husband has Sought Specialist Help,0.67 (𝑥²=254 𝒑<3.43e-57,0.28 (𝑥²=176 𝒑<5.01e-37,0.039 (𝑥²=0.832 𝒑<0.362,0.069 (𝑥²=2.67 𝒑<0.102,0.14 (𝑥²=11.1 𝒑<0.000855,0.1 (𝑥²=5.53 𝒑<0.0187,0.039 (𝑥²=0.845 𝒑<0.358,0.13 (𝑥²=9.53 𝒑<0.00202,0.15 (𝑥²=12 𝒑<0.000533,0.16 (𝑥²=13.8 𝒑<0.000198
Quality Recovery,0.19 (𝑥²=78.6 𝒑<3.5e-16,0.28 (𝑥²=176 𝒑<5.01e-37,0.075 (𝑥²=12.5 𝒑<0.0139,0.053 (𝑥²=6.38 𝒑<0.173,0.057 (𝑥²=7.13 𝒑<0.129,0.1 (𝑥²=23.1 𝒑<0.000122,0.069 (𝑥²=10.5 𝒑<0.0322,0.077 (𝑥²=13.1 𝒑<0.0109,0.095 (𝑥²=20.2 𝒑<0.000448,0.096 (𝑥²=20.5 𝒑<0.000396
Slapped or Thrown Something,0.037 (𝑥²=0.753 𝒑<0.385,0.039 (𝑥²=0.832 𝒑<0.362,0.075 (𝑥²=12.5 𝒑<0.0139,0.56 (𝑥²=173 𝒑<1.84e-39,0.44 (𝑥²=106 𝒑<7.56e-25,0.31 (𝑥²=52.3 𝒑<4.72e-13,0.26 (𝑥²=37.4 𝒑<9.73e-10,0.13 (𝑥²=9.28 𝒑<0.00232,0.16 (𝑥²=13.6 𝒑<0.000226,0.15 (𝑥²=12.8 𝒑<0.000341
Hit with Fist or Something Else,0.07 (𝑥²=2.7 𝒑<0.1,0.069 (𝑥²=2.67 𝒑<0.102,0.053 (𝑥²=6.38 𝒑<0.173,0.56 (𝑥²=173 𝒑<1.84e-39,0.64 (𝑥²=228 𝒑<1.8e-51,0.23 (𝑥²=29.4 𝒑<5.85e-08,0.25 (𝑥²=34.9 𝒑<3.49e-09,0.14 (𝑥²=11 𝒑<0.000917,0.098 (𝑥²=5.35 𝒑<0.0207,0.12 (𝑥²=8.37 𝒑<0.00382
"Kicked, Dragged or Beaten",0.087 (𝑥²=4.25 𝒑<0.0392,0.14 (𝑥²=11.1 𝒑<0.000855,0.057 (𝑥²=7.13 𝒑<0.129,0.44 (𝑥²=106 𝒑<7.56e-25,0.64 (𝑥²=228 𝒑<1.8e-51,0.2 (𝑥²=21.6 𝒑<3.38e-06,0.35 (𝑥²=68.3 𝒑<1.39e-16,0.16 (𝑥²=14.1 𝒑<0.000178,0.15 (𝑥²=12.8 𝒑<0.000352,0.18 (𝑥²=18.1 𝒑<2.06e-05
Choked or Purposely Burnt,0.046 (𝑥²=1.16 𝒑<0.281,0.1 (𝑥²=5.53 𝒑<0.0187,0.1 (𝑥²=23.1 𝒑<0.000122,0.31 (𝑥²=52.3 𝒑<4.72e-13,0.23 (𝑥²=29.4 𝒑<5.85e-08,0.2 (𝑥²=21.6 𝒑<3.38e-06,0.4 (𝑥²=89.2 𝒑<3.52e-21,0.19 (𝑥²=20.8 𝒑<5.19e-06,0.17 (𝑥²=16.1 𝒑<5.94e-05,0.055 (𝑥²=1.69 𝒑<0.193
Threatened or Used a Weapon,0.099 (𝑥²=5.5 𝒑<0.0191,0.039 (𝑥²=0.845 𝒑<0.358,0.069 (𝑥²=10.5 𝒑<0.0322,0.26 (𝑥²=37.4 𝒑<9.73e-10,0.25 (𝑥²=34.9 𝒑<3.49e-09,0.35 (𝑥²=68.3 𝒑<1.39e-16,0.4 (𝑥²=89.2 𝒑<3.52e-21,0.21 (𝑥²=25.4 𝒑<4.68e-07,0.17 (𝑥²=16 𝒑<6.32e-05,0.1 (𝑥²=5.81 𝒑<0.016
Forced Intercourse,0.19 (𝑥²=21 𝒑<4.64e-06,0.13 (𝑥²=9.53 𝒑<0.00202,0.077 (𝑥²=13.1 𝒑<0.0109,0.13 (𝑥²=9.28 𝒑<0.00232,0.14 (𝑥²=11 𝒑<0.000917,0.16 (𝑥²=14.1 𝒑<0.000178,0.19 (𝑥²=20.8 𝒑<5.19e-06,0.21 (𝑥²=25.4 𝒑<4.68e-07,0.38 (𝑥²=82 𝒑<1.37e-19,0.38 (𝑥²=79.3 𝒑<5.31e-19
"Didn't Want Intercourse, but Afraid",0.088 (𝑥²=4.31 𝒑<0.0378,0.15 (𝑥²=12 𝒑<0.000533,0.095 (𝑥²=20.2 𝒑<0.000448,0.16 (𝑥²=13.6 𝒑<0.000226,0.098 (𝑥²=5.35 𝒑<0.0207,0.15 (𝑥²=12.8 𝒑<0.000352,0.17 (𝑥²=16.1 𝒑<5.94e-05,0.17 (𝑥²=16 𝒑<6.32e-05,0.38 (𝑥²=82 𝒑<1.37e-19,0.44 (𝑥²=107 𝒑<3.7e-25
